# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureOpenAIChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta6

### Setting Up the Planner
The planner is located in the `Microsoft.SemanticKernel.Planners.Core` package and requires Orchestration

In [2]:
// Load native plugin into the kernel registry, sharing its functions with prompt templates
var planner = new SequentialPlanner(kernel);

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll give it access to the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "plugins");
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "SummarizePlugin");
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");

Define your ASK. What do you want the Kernel to do?

In [4]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(JsonSerializer.Serialize(originalPlan, new JsonSerializerOptions { WriteIndented = true }));

Original plan:

{
  "state": [],
  "steps": [
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "Date ideas"
        }
      ],
      "outputs": [],
      "next_step_index": 0,
      "name": "Brainstorm",
      "plugin_name": "WriterPlugin",
      "description": "Given a goal or topic description generate a list of ideas",
      "model_settings": null,
      "RequestSettings": {
        "service_id": null,
        "model_id": null,
        "max_tokens": 2000,
        "temperature": 0.5,
        "top_p": 1.0,
        "presence_penalty": 0.0,
        "frequency_penalty": 0.0,
        "stop_sequences": [
          "##END##"
        ]
      }
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "$WriterPlugin.Brainstorm.output"
        }
      ],
      "outputs": [
        "RESULT__poems"
      ],
      "next_step_index": 0,
      "name": "ShortP

As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

The output of each step of the plan gets set as `setContextVariable` which makes it available as `input` to the next plugin.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [5]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";
var shakespeareFunction = kernel.CreateSemanticFunction(skPrompt, "Shakespeare", "ShakespearePlugin", requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.2, TopP = 0.5 });

Let's update our ask using this new plugin.

In [7]:
var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(JsonSerializer.Serialize(newPlan, new JsonSerializerOptions { WriteIndented = true }));

Updated plan:

{
  "state": [],
  "steps": [
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "Date ideas"
        }
      ],
      "outputs": [],
      "next_step_index": 0,
      "name": "Brainstorm",
      "plugin_name": "WriterPlugin",
      "description": "Given a goal or topic description generate a list of ideas",
      "model_settings": null,
      "RequestSettings": {
        "service_id": null,
        "model_id": null,
        "max_tokens": 2000,
        "temperature": 0.5,
        "top_p": 1.0,
        "presence_penalty": 0.0,
        "frequency_penalty": 0.0,
        "stop_sequences": [
          "##END##"
        ]
      }
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "$Brainstorm.output"
        }
      ],
      "outputs": [],
      "next_step_index": 0,
      "name": "ShortPoem",
      "plugin_name": "WriterPlugin",
  

### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [8]:
var originalPlanResult = await kernel.RunAsync(originalPlan);

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.GetValue<string>(), 100));

Original Plan results:

There once was a writer so bright,
Whose ideas took flight day and night.
But when it came time to
write,
Their brain went on strike,
Leaving them in a creative plight!



Now lets execute and print the new plan:

In [9]:
var newPlanResult = await kernel.RunAsync(newPlan);

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult.GetValue<string>());

New Plan results:

In a room full of intense thinkers, a young man named Ethan suggests they take a break and fly kites. Initially met with skepticism, the idea sparks curiosity and excitement, leading the thinkers to experience a carefree moment of joy and connection. This unexpected adventure leaves a lasting impact, bringing a sense of lightheartedness and a deeper bond among the thinkers.
